# RAG асистент

## Устанавливаем зависимости

In [5]:
%pip install -qq langchain langchain-community langchain-qdrant unstructured yandexcloud requests markdown

Note: you may need to restart the kernel to use updated packages.


## Скачиваем документ

In [6]:
import requests
from pathlib import Path

file_link = "https://drive.usercontent.google.com/u/0/uc?id=14LJNTuDM35kGjjpc79VaOFRBPV1ZvKJI&export=download"

response = requests.get(file_link)

if response.status_code == 200:
    print("Загрузка прошла успешно)")
else:
    print(f"Ошибка [ {response.status_code} ]")

Загрузка прошла успешно)


Сохраняем документ на временном диске сессии

In [7]:
# Локальное расположение документа
file_path = Path('devops_book.md')

In [8]:
with open("devops_book.md", "w") as file:
    file.write(response.text)

## Pre-Retrieval

### Чанкование данных

**Импортируем библиотеки**

In [9]:
from langchain_community.document_loaders import (
    TextLoader,
    PyPDFLoader,
    Docx2txtLoader,
    UnstructuredMarkdownLoader
)
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

#### Загрузка документа

In [10]:
def get_loader(file_path: Path):
    suffix = file_path.suffix.lower()

    loaders = {
        ".txt": TextLoader,
        ".pdf": PyPDFLoader,
        ".docx": Docx2txtLoader,
        ".md": UnstructuredMarkdownLoader
    }

    if suffix not in loaders:
        raise ValueError(f"Неподдерживаемый формат файла: {suffix}")

    return loaders[suffix](file_path)

**Загружаем документ**

In [11]:
loader = get_loader(file_path)
document = loader.load()

Проверяем корректность загрузки

In [12]:
print(document[0].page_content)

Научный редактор Николай Корытко Издано с разрешения IT Revolution Press LCC c/o Fletcher & Company и Andrew Nurnberg Associates International Ltd c/o Z AO "Andrew Nurnberg Literary Agency" На русском языке публикуется впервые Благодари м за помощь в подготовке издания Артема Каличкина, Дмитрия Зайцева, Михаи ла Чинкова, Виталия Рыбникова, Дениса Иванова, Валерия Пи лия, Дмитрия Малыхина, Сергея Малютина, А лександра Титова, Дениса Рыбака, Евгения Овчинцева, А лексея К ли мова, Игоря Авдеева Ким, Джен Руководство по DevOps. Как добиться гибкости, надежности и безопасности мирового уровня в технологических компаниях / Джен Ким, Патрик Дебуа, Джон Уиллис, Джез Хамбл ; пер. с англ. И. Лейко и И. Васильева ; [науч. ред. Н. Корытко]. — М. : Манн, Ива- нов и Фербер, 2018. — 512 с. ISBN 978-5-00100-750- Профессиональное движение DevOps зародилось в 2009 году. Его цель — настроить тесные рабочие отношения между разработчиками программ- ного обеспечения и отделами IT-эксплуатации. Внедрение пра

#### Чанкование

**Конфигурационные параметры**

* `chunk_size` - максимальный размер чанка, кол-во символов
* `chunk_overlap` - размер перекрытия чанков (нахлёста), кол-во символов

In [13]:
chunk_size = 1000
chunk_overlap = 500

**Настройка нарезчика данных на чанки**

Будем использовать нарезчик, учитывающий иерархическую структуру документа, т.е. абзаци, главы и т.д.

In [14]:
recursive_character_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

**Чанкование**

In [15]:
chunks = recursive_character_splitter.split_documents(document)

Проверка корректности чанкования

In [16]:
starting_chunk = 4 # Начальный чанк
count_chunks = 2 # Количество выводимых чанков

for i, chunk in enumerate(chunks[starting_chunk:starting_chunk + count_chunks]):
    print(f'---[ {i + starting_chunk} ]---\n {chunk.page_content}')

---[ 4 ]---
 Часть I «Три пути» Введение ..................................................................................... 49

Глава 1 Agile, непрерывная поставка и «три пути» .......................... 55

Глава 2 Первый путь: принципы потока ........................................... 64

Глава 3 Второй путь: принципы обратной связи............................ 80

Глава 4 Третий путь: принципы непрерывного обучения и экспериментирования ..................................... 92

Часть II Откуда начать Введение ..................................................................................... 109

Глава 5 Как выбрать стартовый поток создания ценности ......... 111

Глава 6 Основные сведения о работе в потоке создания ценности, превращении его в прозрачный и расширении на всю организацию .................................. 124

Глава 7 Как проектировать организацию и ее архитектуру, не забывая о законе Конвея ................................................... 144
---[ 5 ]---
 Час

## Retrieval

**Импорт библиотек**

In [17]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain_qdrant.qdrant import QdrantVectorStore

from langchain_community.embeddings import YandexGPTEmbeddings

#from google.colab import userdata

### Развёртывание векторной БД

**Конфигурационные параметры**

* `collection_name` - наименование коллекции в БД, в которой будут храниться загруженные данные.
* `vector_size` - количество значений в векторном представлении, его длина. У используемых эмбеддинговых моделей от Яндекса максимальная длина 256, поэтому здесь также её придерживаемся.
* `distance` - метрика для определения расстояния между векторами. Лучше представлять, как обыкновенное расстояние между точками в пространстве.

In [18]:
collection_name = "devopsconf_collection"
vector_size = 256
distance = 'Cosine'

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")

YANDEX_API_KEY = os.getenv("YANDEX_API_KEY")
YANDEX_FOLDER_ID = os.getenv("YANDEX_FOLDER_ID")

**Подключению к облачному кластеру**

In [25]:
qdrant_client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY,
)

**Создание коллекции данных в кластере**

Создание коллекции

In [17]:
if collection_name not in [collection.name for collection in qdrant_client.get_collections().collections]:
    qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=vector_size,
        distance=Distance(distance)
    )
)

**Создание векторного хранилища**

Для создания эмбедингов будем использовать модель YandexGPTEmbeddings

In [18]:
# Загрузка модели для создания эмбеддингов
yandex_embedding = YandexGPTEmbeddings(
    api_key=YANDEX_API_KEY,
    folder_id=YANDEX_FOLDER_ID
)

# Инициализация векторного хранилища
vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name=collection_name,
    embedding=yandex_embedding
)

Retrying langchain_community.embeddings.yandex._embed_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised _InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAUTHENTICATED
	details = "Unknown api key 'AQVN****NRfM (BDAB17FB)'"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:158.160.54.160:443 {created_time:"2025-04-13T13:09:08.802454+02:00", grpc_status:16, grpc_message:"Unknown api key \'AQVN****NRfM (BDAB17FB)\'"}"
>.
Retrying langchain_community.embeddings.yandex._embed_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised _InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAUTHENTICATED
	details = "Unknown api key 'AQVN****NRfM (BDAB17FB)'"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:158.160.54.160:443 {grpc_message:"Unknown api key \'AQVN****NRfM (BDAB17FB)\'", grpc_status:16, created_time:"2025-04-13T13:09:10.535813+02:00"}"
>.
Retrying lan

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAUTHENTICATED
	details = "Unknown api key 'AQVN****NRfM (BDAB17FB)'"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:158.160.54.160:443 {grpc_message:"Unknown api key \'AQVN****NRfM (BDAB17FB)\'", grpc_status:16, created_time:"2025-04-13T13:09:43.355577+02:00"}"
>

### Добавление базы знаний в векторную БД

#### Принцип работы модели для создания эмбеддингов

Искомый текст:

In [ ]:
text = chunks[0].page_content
print(text)

Текст преобразованный в эмбеддинг (векторное представление):

In [ ]:
print(yandex_embedding.embed_query(text))

**Добавляем чанки в векторную БД**

In [ ]:
vector_store.add_documents(chunks)

#### Поиск в векторной БД

**Конфигурационные параметры**

* `k` - максимальное количетсво возвращаемых валидных чанков. Другими словами - это топ `k` самых близких к запросу чанков.
* `query` - пользовательский запрос.

In [ ]:
k = 5
query = "Перескажи в краце о чём говорится в предисловии к русскому изданию."

**Поиск валидных чанков**

In [ ]:
searching_chunks = vector_store.similarity_search(
    query=query,
    k=k
)

Топ `k` наиболее близких к запросу чанков

In [ ]:
for i, searching_chunk in enumerate(searching_chunks):
    print(f'{i + 1}. {searching_chunk.page_content}')

## Generative

**Импорт библиотек**

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import YandexGPT

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

**Конфигурационные параметры**
* `model_name` - наименование модели
* `temperature` - температура - отвечает за степень галлюционирования модели, т.е. добавления в ответы несвязных данных, выбросов и т.д. Данный параметр примимает занчения в промежутке `[0, 1]`
* `max_tokens` - максимальное количество токенов, доступных для полного цикла работы модели, т.е. сумма колличества токенов из входного запроса плюс - в ответе. Обыно у моделей данный параметр может принимать различные максимумы, поэтому необходимо читать документацию конкретного экземпляра.

In [ ]:
model_name = 'yandexgpt'
temperature = 0.7
max_tokens = 8000

**Загрузка LLM**



In [ ]:
yandexgpt = YandexGPT(
    api_key=YANDEX_API_KEY,
    folder_id=YANDEX_FOLDER_ID,
    model_name=model_name,
    temperature=temperature,
    max_tokens=max_tokens
)

**Создание шаблона запроса**

In [ ]:
system_prompt ="""
**Роль**
Ты — DevOps с большим стажем, который недавно написал документацию и не любит долгих разговоров.

**Инструкции**
1. Используй приведённый контекст для ответа на вопрос.
2. Если ты не можешь найти ответ в контексте, так и скажи: 'В документе отсутствуют данные для формирования ответа.', не пытайся придумать ответ.
3. Твои высказывания должны быть связными по смыслу и чётко доносить мысль из контекста.
4. Сфомируй ответ простым текстом, не более 10 слов.

**Контекст**
{context}
"""

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', system_prompt),
        ('human', '{input}')
    ]
)

**Финализация RAG pipeline`а**

In [ ]:
question_answer_chain = create_stuff_documents_chain(
    yandexgpt,
    prompt
)

chain = create_retrieval_chain(
    vector_store.as_retriever(),
    question_answer_chain
)

Отправляем запрос:

In [ ]:
print(query)

In [ ]:
answer = chain.invoke({"input": query})

In [ ]:
print(answer["answer"])

### Общение с RAG-ассистентом

In [ ]:
def get_answer(query):
    k = 20

    searching_chunks = vector_store.similarity_search(
        query=query,
        k=k
    )

    system_prompt ="""
        **Роль**
        Ты — DevOps с большим стажем, который недавно написал документацию и не любит долгих разговоров.

        **Инструкции**
        1. Используй приведённый контекст для ответа на вопрос.
        2. Если ты не можешь найти ответ в контексте, так и скажи: 'В документе отсутствуют данные для формирования ответа.', не пытайся придумать ответ.
        3. Твои высказывания должны быть связными по смыслу и чётко доносить мысль из контекста.

        **Контекст**
        {context}
    """

    prompt = ChatPromptTemplate.from_messages(
        [
            ('system', system_prompt),
            ('human', '{input}')
        ]
    )

    question_answer_chain = create_stuff_documents_chain(
        yandexgpt,
        prompt
    )

    chain = create_retrieval_chain(
        vector_store.as_retriever(),
        question_answer_chain
    )

    answer = chain.invoke({"input": query})

    return answer["answer"]

**Пример 1**

In [ ]:
query = "В чём цель данной книги?"

In [ ]:
print(get_answer(query))

**Пример 2**

In [ ]:
query = "Какие были предпосылки к появлению DevOps, как профессии?"

In [ ]:
print(get_answer(query))